In [ ]:
console.log("Starting script...");
const admin = require('firebase-admin');

try {
    const serviceAccount = require('../firebase-service-account.json');
    if (admin.apps.length === 0) {
        admin.initializeApp({
            credential: admin.credential.cert(serviceAccount)
        });
    }
    console.log("Firebase Admin Initialized Successfully");
} catch (error) {
    console.error("Error initializing Firebase:", error);
}

const db = admin.firestore();
console.log("DB instance created");

In [ ]:
// 2. Define Target User and Distributor IDs
const STORE_ID = 'U85Mqj9IkDYHTEFz0Pb7Yo1J5S93';
const EXPECTED_DISTRIBUTOR_ID = 'DIST-ECU-02';

console.log(`Checking Store ID: ${STORE_ID}`);
console.log(`Expected Distributor ID: ${EXPECTED_DISTRIBUTOR_ID}`);

In [ ]:
// 3. Query Recent Invoices for Store User
const invoicesSnapshot = await db.collection('invoices')
    .where('storeId', '==', STORE_ID)
    .orderBy('createdAt', 'desc')
    .limit(1)
    .get();

if (invoicesSnapshot.empty) {
    console.log('No invoices found for this store.');
} else {
    const invoiceDoc = invoicesSnapshot.docs[0];
    const invoiceData = invoiceDoc.data();
    console.log(`Found invoice ID: ${invoiceDoc.id}`);
    console.log('Invoice Data:', JSON.stringify(invoiceData, null, 2));
    
    // Check critical fields
    console.log('--- Field Check ---');
    console.log(`status: '${invoiceData.status}' (Expected: 'pending')`);
    console.log(`distributorId: '${invoiceData.distributorId}' (Expected: '${EXPECTED_DISTRIBUTOR_ID}')`);
    console.log(`storeId: '${invoiceData.storeId}' (Expected: '${STORE_ID}')`);
    
    if (invoiceData.distributorId !== EXPECTED_DISTRIBUTOR_ID) {
        console.error(`❌ ERROR: distributorId mismatch! Expected '${EXPECTED_DISTRIBUTOR_ID}', found '${invoiceData.distributorId}'`);
        
        // Check Store Data to see why it might be missing
        const storeDoc = await db.collection('stores').doc(STORE_ID).get();
        if (storeDoc.exists) {
            console.log('Store Data distributorId:', storeDoc.data().distributorId);
        } else {
            console.log('Store document not found!');
        }
    } else {
        console.log('✅ distributorId matches.');
    }
}

In [ ]:
// 5. Simulate Distributor Query
// The query used in the app is:
// where('countryId', '==', currentUser!.countryId),
// where('distributorId', '==', currentUser!.distributorId),
// where('status', '==', 'pending'),
// orderBy('createdAt', 'desc')

// We need to fetch the countryId first from the store or invoice to exact match
// Let's assume countryId is 'ECU' based on distributorId 'DIST-ECU-02' but let's check invoice data
const invoice = invoicesSnapshot.docs[0].data();
const COUNTRY_ID = invoice.countryId;

console.log(`Simulating query for Country: ${COUNTRY_ID}, Distributor: ${EXPECTED_DISTRIBUTOR_ID}`);

try {
    const distQuerySnapshot = await db.collection('invoices')
        .where('countryId', '==', COUNTRY_ID)
        .where('distributorId', '==', EXPECTED_DISTRIBUTOR_ID)
        .where('status', '==', 'pending')
        .orderBy('createdAt', 'desc')
        .get();

    console.log(`Query returned ${distQuerySnapshot.size} documents.`);
    
    const found = distQuerySnapshot.docs.find(d => d.id === invoicesSnapshot.docs[0].id);
    if (found) {
        console.log(`✅ Invoice ${found.id} IS in the results.`);
    } else {
        console.log(`❌ Invoice ${invoicesSnapshot.docs[0].id} is NOT in the results.`);
        console.log("Possible reasons: Index missing, or field mismatch.");
    }
} catch (error) {
    console.error("Query failed:", error);
    if (error.code === 9) { // FAILED_PRECONDITION
        console.log("This usually means a missing index.");
    }
}